In [1]:
import numpy as np
from Lib.utils import mkdirs
from Lib.resnet_model import Resnet3DBuilder
import Lib.image as img
from Lib.HistoryGraph import HistoryGraph

In [2]:
from tensorflow.keras.optimizers import SGD

In [3]:
from keras.callbacks import ModelCheckpoint

In [4]:
target_size=(64,96)
nb_frames=16
skip=1
batch_size=64
input_shape=(nb_frames,)+ target_size+ (3,)
nb_classes=27

In [5]:
workers=8
use_multiprocessing = False 
max_queue_size = 20

In [6]:
import pandas as pd
import numpy as np

import sys

sys.executable

class DataLoader():
  def __init__(self,path_vid,path_labels,path_train=None,path_val=None,path_test=None):
    self.path_vid=path_vid
    self.path_labels=path_labels
    self.path_train=path_train
    self.path_val=path_val
    self.path_test=path_test

    self.get_labels(path_labels)

    if self.path_train:
      self.train_df = self.load_video_labels(self.path_train)
    if self.path_val:
      self.val_df =self.load_video_labels(self.path_val)
    if self.path_test:
      self.test_df=self.load_video_labels(self.path_test,mode="input")

  def get_labels(self,path_labels):
    self.labels_df =pd.read_csv(path_labels,names=['labels'])
    #extract labels from data frames
    self.labels=[str(label[0]) for label in self.labels_df.values]
 #   print(self.labels)
    self.n_labels=len(self.labels)
 #   print(self.n_labels)
    #create dict. to convert label to int and backwards
    self.label_to_int = dict(zip(self.labels,range(self.n_labels)))
#    print(self.label_to_int)
    self.int_to_labels = dict(enumerate(self.labels))
#    print(self.int_to_labels)
    dictionary = {0:self.labels}

  def load_video_labels(self,path_subset,mode='label'):
    if mode=='input':
        names=['video_id']
    elif mode=='label':
        names=['video_id','label']

    df =pd.read_csv(path_subset,sep=';',names=names)

    if mode=='label':
      df=df[df.label.isin(self.labels)]
 #   print(df)
    return df
  #def catgerocial_to_label(self):
    

In [7]:
#self.labels_df = pd.read_csv(path_labels, names=['label'])  # Or specific encoding you find

In [8]:
data_root = r"C:\Users\Asus1\Downloads\archive"

In [9]:
import os

In [10]:
csv_labels= r"C:\Users\Asus1\Downloads\archive\new_jester_v1.csv"
csv_train= r"C:\Users\Asus1\Downloads\archive\new_train.csv"
csv_test= r"C:\Users\Asus1\Downloads\archive\new_test.csv"
csv_val=r"C:\Users\Asus1\Downloads\archive\new_validation.csv"
data_test_vid=r"C:\Users\Asus1\Downloads\archive\Test"

model_name='resent_3d_model'
data_model=r"c"

In [11]:
from os import path

path_model=os.path.join(data_root,data_model,model_name)

path_vid = os.path.join(data_root,data_test_vid)
path_labels= os.path.join(data_root,csv_labels) 
path_train= os.path.join(data_root, csv_train) 
path_val=os.path.join(data_root, csv_val)
path_test = os.path.join(data_root, csv_test) 

In [12]:
data = DataLoader(path_vid,path_labels,path_train,path_val)

In [13]:
def mkdirs(folder,permission):
    if not os.path.exists(folder):
        try:
            original_umask=os.umask(0)
            os.makedirs(folder,permission,exist_ok=True)
        finally:
            os.umask(original_umask)

In [14]:
#data = DataLoader(path_vid,path_labels,path_train,path_val)
mkdirs(path_model,0o755)
mkdirs(os.path.join(path_model,"graphs"),0o755)

In [15]:
gen=img.ImageDataGenerator()
gen_train=gen.flow_video_from_dataframe(data.train_df,path_vid,path_classes=path_labels,x_col='video_id',y_col='labels',target_size=target_size, batch_size=batch_size,nb_frames=nb_frames,skip=skip,has_ext=True)
gen_val=gen.flow_video_from_dataframe(data.val_df,path_vid,path_classes=path_labels,x_col='video_id',y_col='labels',target_size=target_size,batch_size=batch_size,nb_frames=nb_frames,skip=skip,has_ext=True)


Found 48621 video folders belonging to 27 classes.
Found 6785 video folders belonging to 27 classes.


In [16]:
resnet_model=Resnet3DBuilder.build_resnet_101(input_shape,nb_classes,drop_rate=0.5)
optimizer=SGD(lr=0.01,momentum=0.9,decay=0.0001,nesterov=False)
resnet_model.compile(optimizer=optimizer,loss="categorical_crossentropy",metrics=["accuracy"])
model_file=os.path.join(path_model,'resnetmodel.hdf5')

C:\Users\Asus1\anaconda3\envs\dhgr\lib\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [17]:
model_checkpointer=ModelCheckpoint(model_file,monitors='val_acc',verbose=1,save_best_only=True,mode=max)


In [18]:
history_graph=HistoryGraph(model_path_name=os.path.join(path_model,"graphs"))

In [19]:
nb_sample_train=data.train_df["video_id"].size
nb_sample_val=data.val_df["video_id"].size

In [20]:
import math
resnet_model.fit_generator(
generator=gen_train,
steps_per_epoch=math.ceil(nb_sample_train / batch_size),
epochs=10,
validation_data=gen_val,
validation_steps=30,
shuffle=False,
verbose=1,
workers=workers,
max_queue_size=max_queue_size,
use_multiprocessing=use_multiprocessing,
callbacks=[model_checkpointer, history_graph])